Merging the xT, VAEP and Expected VAEP across chains.

In [1]:
import pandas as pd
import numpy as np
import joblib

import sys
sys.path.append("..")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/expected-threat")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp-vaep")

from config import chain_file_path

import xthreat as xthreat
import vaep.domain.preprocessing.preprocessing as vaep
import vaep.domain.preprocessing.formula as vaep_formula
import exp_vaep.domain.preprocessing.preprocessing as exp_vaep
import exp_vaep.domain.preprocessing.formula as exp_vaep_formula

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
rounds_list = ["202320"]

In [3]:
chains = pd.read_csv(chain_file_path)
round_chains_list = []
for round_id in rounds_list:
    round_chain = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/round_chains_"+round_id+".csv")
    round_chains_list.append(round_chain)
chains = pd.concat([chains]+round_chains_list, axis=0)
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN


In [4]:
number_of_rows = chains.shape[0]

Get Expected Threat

In [5]:
def convert_chain_to_spadl(chains):
    
    xthreat_data = chains.copy()

    xthreat_data = xthreat_data.dropna(subset=['Player'])
    xthreat_data = xthreat_data[~(xthreat_data['Description']=="Kick Into F50")]
    xthreat_data = xthreat_data[~(xthreat_data['Description']=="Goal")]

    xthreat_data['end_x'] = xthreat_data.groupby('Match_ID')['x'].shift(-1).fillna(0)
    xthreat_data['end_y'] = xthreat_data.groupby('Match_ID')['y'].shift(-1).fillna(0)
    
    xthreat_data['Receiver'] = xthreat_data.groupby('Match_ID')['Player'].shift(-1).fillna(0)
    xthreat_data['ReceiverTeam'] = xthreat_data.groupby('Match_ID')['Team'].shift(-1).fillna(0)

    xthreat_data['type_name'] = np.where(xthreat_data['Description'] == "Handball", "Handball",
                                        np.where((xthreat_data['Description'] == 'Kick') & (xthreat_data['Shot_At_Goal']==True), "Shot",
                                                np.where(xthreat_data['Description']=='Kick', "Kick",
                                                            "Carry")))
    xthreat_data['outcome_name'] = (xthreat_data['type_name'] == "Carry") | (xthreat_data['Disposal'] == "effective")
    xthreat_data['goal'] = (xthreat_data['type_name'] == "Shot") & (xthreat_data['Final_State'] == "goal")
    xthreat_data['shoot'] = xthreat_data['type_name'] == "Shot"
    xthreat_data['move'] = xthreat_data['type_name'] != "Shot"

    xthreat_data['Year'] = xthreat_data['Match_ID'].apply(lambda x: int(x[:4]))
    xthreat_data = xthreat_data[['Match_ID', 'Chain_Number', 'Order', 'Year', 'Team', 'Player', 'ReceiverTeam', 'Receiver', 'type_name', 'outcome_name', 'shoot', 'move', 'goal', 'x', 'y', 'end_x', 'end_y']]
    
    return xthreat_data

In [6]:
xthreat_actions = convert_chain_to_spadl(chains)
xthreat_actions.head()

,Match_ID,Chain_Number,Order,Year,Team,Player,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,x,y,end_x,end_y
1,202101_BrisbaneLions_Sydney,1,2.0,2021,Brisbane Lions,Dayne Zorko,Brisbane Lions,Dayne Zorko,Carry,True,False,True,False,8.0,-5.0,9.0,-6.0
2,202101_BrisbaneLions_Sydney,1,3.0,2021,Brisbane Lions,Dayne Zorko,Sydney,Oliver Florent,Handball,False,False,True,False,9.0,-6.0,11.0,-7.0
3,202101_BrisbaneLions_Sydney,1,4.0,2021,Sydney,Oliver Florent,Sydney,Oliver Florent,Carry,True,False,True,False,11.0,-7.0,12.0,-5.0
4,202101_BrisbaneLions_Sydney,1,5.0,2021,Sydney,Oliver Florent,Sydney,George Hewett,Handball,True,False,True,False,12.0,-5.0,14.0,-2.0
5,202101_BrisbaneLions_Sydney,1,6.0,2021,Sydney,George Hewett,Sydney,George Hewett,Carry,True,False,True,False,14.0,-2.0,22.0,-2.0


In [7]:
xTModel = xthreat.load_model("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/expected-threat/models/xT_v02.json")

xT Created (Successful Actions)

In [8]:
move_actions = xthreat.get_successful_move_actions(xthreat_actions)
move_actions['xT_created'] = xTModel.rate(move_actions)
move_actions.head()

,Match_ID,Chain_Number,Order,Year,Team,Player,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,x,y,end_x,end_y,xT_created
1,202101_BrisbaneLions_Sydney,1,2.0,2021,Brisbane Lions,Dayne Zorko,Brisbane Lions,Dayne Zorko,Carry,True,False,True,False,8.0,-5.0,9.0,-6.0,0.005595
3,202101_BrisbaneLions_Sydney,1,4.0,2021,Sydney,Oliver Florent,Sydney,Oliver Florent,Carry,True,False,True,False,11.0,-7.0,12.0,-5.0,0.000000
4,202101_BrisbaneLions_Sydney,1,5.0,2021,Sydney,Oliver Florent,Sydney,George Hewett,Handball,True,False,True,False,12.0,-5.0,14.0,-2.0,0.007561
5,202101_BrisbaneLions_Sydney,1,6.0,2021,Sydney,George Hewett,Sydney,George Hewett,Carry,True,False,True,False,14.0,-2.0,22.0,-2.0,0.020650
7,202101_BrisbaneLions_Sydney,1,8.0,2021,Brisbane Lions,Hugh McCluggage,Brisbane Lions,Hugh McCluggage,Carry,True,False,True,False,6.0,-27.0,11.0,-26.0,0.001855


xT Denied (Unsuccessful Actions + Team Changes)

In [9]:
move_unsuccessful_actions = xthreat.get_move_actions(xthreat_actions)
move_unsuccessful_actions = move_unsuccessful_actions[move_unsuccessful_actions['outcome_name'] == False]
move_unsuccessful_actions = move_unsuccessful_actions[(move_unsuccessful_actions['Team'] != move_unsuccessful_actions['ReceiverTeam'])]
move_unsuccessful_actions['outcome_name'] = True
move_unsuccessful_actions['xT_denied'] = xTModel.rate(move_unsuccessful_actions)
move_unsuccessful_actions['outcome_name'] = False
move_unsuccessful_actions.head()

,Match_ID,Chain_Number,Order,Year,Team,Player,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,x,y,end_x,end_y,xT_denied
2,202101_BrisbaneLions_Sydney,1,3.0,2021,Brisbane Lions,Dayne Zorko,Sydney,Oliver Florent,Handball,False,False,True,False,9.0,-6.0,11.0,-7.0,0.000000
6,202101_BrisbaneLions_Sydney,1,7.0,2021,Sydney,George Hewett,Brisbane Lions,Hugh McCluggage,Kick,False,False,True,False,22.0,-2.0,6.0,-27.0,-0.040413
29,202101_BrisbaneLions_Sydney,3,30.0,2021,Sydney,Luke Parker,Brisbane Lions,Tom Fullarton,Handball,False,False,True,False,-46.0,-20.0,-44.0,-25.0,0.004496
34,202101_BrisbaneLions_Sydney,4,35.0,2021,Brisbane Lions,Lincoln McCarthy,Sydney,Luke Parker,Kick,False,False,True,False,40.0,25.0,-54.0,19.0,-0.126150
46,202101_BrisbaneLions_Sydney,5,47.0,2021,Sydney,Luke Parker,Brisbane Lions,Grant Birchall,Kick,False,False,True,False,-8.0,26.0,11.0,19.0,0.024862


In [10]:
threat_actions = pd.concat([move_actions, move_unsuccessful_actions], axis=0).sort_values(by=['Match_ID', 'Chain_Number', "Order"])

In [11]:
chains = chains.merge(threat_actions[['Match_ID', 'Chain_Number', 'Order', "ReceiverTeam", "Receiver", "type_name", 'outcome_name', 'shoot', 'move', 'goal', 'end_x', 'end_y', "xT_created", "xT_denied"]], how = "left", on = ['Match_ID', 'Chain_Number', 'Order'])
assert chains.shape[0] == number_of_rows
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,end_x,end_y,xT_created,xT_denied
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Brisbane Lions,Dayne Zorko,Carry,True,False,True,False,9.0,-6.0,0.005595,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Handball,False,False,True,False,11.0,-7.0,NaN,0.0
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Carry,True,False,True,False,12.0,-5.0,0.000000,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,George Hewett,Handball,True,False,True,False,14.0,-2.0,0.007561,NaN


Score VAEP

In [12]:
schema_chains = vaep.convert_chains_to_schema(chains)
gamestate_features = vaep.create_gamestate_features(schema_chains)

In [13]:
vaep_modelling_data = pd.concat([schema_chains, gamestate_features], axis=1)

Load models

In [14]:
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/model_outputs/models/"
scoring_model = "vaep_scoring_v1.joblib"
conceding_model = "vaep_conceding_v1.joblib"

In [15]:
score_model = joblib.load(model_file_path + "/" + scoring_model)
concede_model = joblib.load(model_file_path + "/" + conceding_model)

Make predictions

In [16]:
model_features = score_model.xgb_model.get_booster().feature_names

In [17]:
vaep_features = vaep_modelling_data[model_features]

In [18]:
schema_chains['scores'] = score_model.predict_proba(vaep_features, calibrate=True)
schema_chains['concedes'] = concede_model.predict_proba(vaep_features, calibrate=True)

Compute VAEP

In [19]:
match_list = list(schema_chains['match_id'].unique())
match_vaep_list = []
for match in match_list:
    match_chains = schema_chains[schema_chains['match_id'] == match]
    v = vaep_formula.value(match_chains, match_chains['scores'], match_chains['concedes'])
    match_vaep_list.append(v)
    
vaep_values = pd.concat(match_vaep_list, axis=0)

In [20]:
chain_vaep_data = pd.concat([schema_chains, vaep_values], axis=1)
chain_vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value
1,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.052962,0.009920,0.000000,-0.000000,0.000000
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.022556,0.050321,-0.030406,-0.040401,-0.070807
3,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.074422,0.008536,0.024101,0.014020,0.038121
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.084299,0.013648,0.009877,-0.005111,0.004766
5,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.215944,0.012438,0.131645,0.001209,0.132854


Score Expected VAEP

Get Expected Scores

In [21]:
expected_score_version = 4
model_location = '/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/'
expected_goal_set_version = 7
expected_behind_set_version = 5
expected_miss_set_version = 4

expected_goal_open_version = 7
expected_behind_open_version = 6
expected_miss_open_version = 5

In [22]:
expected_goal_set_model = joblib.load(model_location+"expected_goal_set_v"+str(expected_goal_set_version)+".joblib")
expected_behind_set_model = joblib.load(model_location+"expected_behind_set_v"+str(expected_behind_set_version)+".joblib")
expected_miss_set_model = joblib.load(model_location+"expected_miss_set_v"+str(expected_miss_set_version)+".joblib")

expected_goal_open_model = joblib.load(model_location+"expected_goal_open_v"+str(expected_goal_open_version)+".joblib")
expected_behind_open_model = joblib.load(model_location+"expected_behind_open_v"+str(expected_behind_open_version)+".joblib")
expected_miss_open_model = joblib.load(model_location+"expected_miss_open_v"+str(expected_miss_open_version)+".joblib")

In [23]:
goal_set_features = expected_goal_set_model.xgb_model.get_booster().feature_names
behind_set_features = expected_behind_set_model.xgb_model.get_booster().feature_names
miss_set_features = expected_miss_set_model.xgb_model.get_booster().feature_names

goal_open_features = expected_goal_open_model.xgb_model.get_booster().feature_names
behind_open_features = expected_behind_open_model.xgb_model.get_booster().feature_names
miss_open_features = expected_miss_open_model.xgb_model.get_booster().feature_names

In [24]:
chains[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(chains['Initial_State'])

In [25]:
chains['Event_Type1'] = chains['Description'].shift(1)
shots = chains[chains['Shot_At_Goal'] == True]
shots['Set_Shot'] = shots['Event_Type1'].apply(lambda x: ("Mark" in x) or ("Free" in x))
set_shots = shots[shots['Set_Shot']]
open_shots = shots[~shots['Set_Shot']]

In [26]:
set_shots = exp_vaep.expected_score_feature_engineering(set_shots)
open_shots = exp_vaep.expected_score_feature_engineering(open_shots)

In [27]:
set_shots['xGoals'] = expected_goal_set_model.predict_proba(set_shots[goal_set_features], calibrate=True)
set_shots['xBehinds'] = expected_behind_set_model.predict_proba(set_shots[behind_set_features], calibrate=True)
set_shots['xMiss'] = expected_miss_set_model.predict_proba(set_shots[miss_set_features], calibrate=True)

open_shots['xGoals'] = expected_goal_open_model.predict_proba(open_shots[goal_open_features], calibrate=True)
open_shots['xBehinds'] = expected_behind_open_model.predict_proba(open_shots[behind_open_features], calibrate=True)
open_shots['xMiss'] = expected_miss_open_model.predict_proba(open_shots[miss_open_features], calibrate=True)

In [28]:
exp_shots = pd.concat([set_shots, open_shots], axis=0)
exp_shots = exp_shots.sort_values(by = ['Match_ID', "Chain_Number", "Order"])
shots = shots.merge(exp_shots[['Chain_Number', 'Order', 'Match_ID', 'xGoals', 'xBehinds', 'xMiss']], how = "left", on = ['Chain_Number', 'Order', 'Match_ID'])

In [29]:
shots['xGoals_normalised'] = shots['xGoals'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xBehinds_normalised'] = shots['xBehinds'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xMiss_normalised'] = shots['xMiss'] / (shots['xGoals'] + shots['xBehinds'] + shots['xMiss'])
shots['xScore'] = shots['xGoals_normalised']*6 + shots['xBehinds_normalised']

In [30]:
chains = chains.merge(shots[['Match_ID', 'Chain_Number', 'Order', 'xScore', 'xGoals_normalised']], how="left", on = ['Match_ID', 'Chain_Number', 'Order'])
chains[['xScore', 'xGoals_normalised']] = chains[['xScore', 'xGoals_normalised']].fillna(0)
assert chains.shape[0] == number_of_rows
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,end_x,end_y,xT_created,xT_denied,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type1,xScore,xGoals_normalised
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,None,0.0,0.0
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Brisbane Lions,Dayne Zorko,Carry,True,False,True,False,9.0,-6.0,0.005595,NaN,0,1,0,0,0,Centre Bounce,0.0,0.0
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Handball,False,False,True,False,11.0,-7.0,NaN,0.0,0,1,0,0,0,Hard Ball Get,0.0,0.0
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Carry,True,False,True,False,12.0,-5.0,0.000000,NaN,0,1,0,0,0,Handball,0.0,0.0
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,George Hewett,Handball,True,False,True,False,14.0,-2.0,0.007561,NaN,0,1,0,0,0,Loose Ball Get,0.0,0.0


Preprocess for Expected VAEP

In [31]:
schema_chains = exp_vaep.convert_chains_to_schema(chains)
gamestate_features = exp_vaep.create_gamestate_features(schema_chains)

In [32]:
schema_chains.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore
1,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.0
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.0
3,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.0
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.0
5,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.0


In [33]:
exp_vaep_modelling_data = pd.concat([schema_chains, gamestate_features], axis=1)

Load models

In [34]:
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp_vaep/model_outputs/models/"
scoring_model = "exp_vaep_scoring_v1.joblib"
conceding_model = "exp_vaep_conceding_v1.joblib"

In [35]:
exp_score_model = joblib.load(model_file_path + "/" + scoring_model)
exp_concede_model = joblib.load(model_file_path + "/" + conceding_model)

In [36]:
exp_model_features = exp_score_model.get_booster().feature_names

In [37]:
exp_vaep_features = exp_vaep_modelling_data[exp_model_features]

Make predictions

In [38]:
schema_chains['scores'] = np.clip(exp_score_model.predict(exp_vaep_features), 0, 6)
schema_chains['concedes'] = np.clip(exp_concede_model.predict(exp_vaep_features), 0, 6)

In [39]:
schema_chains['scores'].describe()

count    961249.000000
mean          0.814941
std           0.848606
min           0.000000
25%           0.233351
50%           0.596003
75%           1.083445
max           6.000000
Name: scores, dtype: float64

Compute VAEP

In [40]:
match_list = list(schema_chains['match_id'].unique())
match_exp_vaep_list = []
for match in match_list:
    match_chains = schema_chains[schema_chains['match_id'] == match]
    v = exp_vaep_formula.value(match_chains, match_chains['scores'], match_chains['concedes'])
    match_exp_vaep_list.append(v)
    
exp_vaep_values = pd.concat(match_exp_vaep_list, axis=0)

In [41]:
chain_exp_vaep_data = pd.concat([schema_chains, exp_vaep_values], axis=1)
chain_exp_vaep_data.head(12)

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,scores,concedes,offensive_value,defensive_value,vaep_value
1,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.000000,0.246684,0.158121,0.000000,-0.000000,0.000000
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.000000,0.000000,0.362654,-0.246684,-0.204533,-0.451217
3,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.000000,0.668677,0.081827,0.306023,-0.081827,0.224196
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.000000,0.456306,0.099139,-0.212371,-0.017312,-0.229683
5,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.000000,1.176206,0.069984,0.719900,0.029155,0.749055
6,202101_BrisbaneLions_Sydney,1,7.0,1,31.0,31.0,Sydney,George Hewett,-22.0,2.0,6.0,-27.0,Kick,clanger,0.000000,0.000000,0.438838,-1.176206,-0.368854,-1.545059
7,202101_BrisbaneLions_Sydney,1,8.0,1,36.0,36.0,Brisbane Lions,Hugh McCluggage,6.0,-27.0,11.0,-26.0,Gather,effective,0.000000,1.003286,0.063453,0.564449,-0.063453,0.500995
8,202101_BrisbaneLions_Sydney,1,9.0,1,37.0,37.0,Brisbane Lions,Hugh McCluggage,11.0,-26.0,18.0,-23.0,Handball,effective,0.000000,0.499200,0.037909,-0.504086,0.025544,-0.478542
9,202101_BrisbaneLions_Sydney,1,10.0,1,38.0,38.0,Brisbane Lions,Zac Bailey,18.0,-23.0,26.0,-21.0,Carry,effective,0.000000,1.979520,0.048713,1.480319,-0.010804,1.469515
10,202101_BrisbaneLions_Sydney,1,11.0,1,39.0,39.0,Brisbane Lions,Zac Bailey,26.0,-21.0,26.0,-21.0,Shot,effective,1.727293,2.211809,0.129209,0.232290,-0.080495,0.151794


In [42]:
chain_exp_vaep_data = chain_exp_vaep_data.rename(columns = {
    "scores":"exp_scores",
    "concedes":"exp_concedes",
    "offensive_value":"exp_offensive_value",
    "defensive_value":"exp_defensive_value",
    "vaep_value":"exp_vaep_value"})
chain_exp_vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
1,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.0,0.246684,0.158121,0.000000,-0.000000,0.000000
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.0,0.000000,0.362654,-0.246684,-0.204533,-0.451217
3,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.0,0.668677,0.081827,0.306023,-0.081827,0.224196
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.0,0.456306,0.099139,-0.212371,-0.017312,-0.229683
5,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.0,1.176206,0.069984,0.719900,0.029155,0.749055


In [43]:
chain_vaep_data = chain_vaep_data.merge(chain_exp_vaep_data[['match_id', 'chain_number', 'order', 'exp_scores', 'exp_concedes', 'exp_offensive_value', 'exp_defensive_value', "exp_vaep_value"]], how = "left", on=['match_id', 'chain_number', 'order'])
chain_vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
0,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.052962,0.009920,0.000000,-0.000000,0.000000,0.246684,0.158121,0.000000,-0.000000,0.000000
1,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.022556,0.050321,-0.030406,-0.040401,-0.070807,0.000000,0.362654,-0.246684,-0.204533,-0.451217
2,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.074422,0.008536,0.024101,0.014020,0.038121,0.668677,0.081827,0.306023,-0.081827,0.224196
3,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.084299,0.013648,0.009877,-0.005111,0.004766,0.456306,0.099139,-0.212371,-0.017312,-0.229683
4,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.215944,0.012438,0.131645,0.001209,0.132854,1.176206,0.069984,0.719900,0.029155,0.749055


In [44]:
chains = chains.merge(chain_vaep_data[['match_id', 'chain_number', 'order', "overall_seconds", 'action_type', 'outcome_type', 'scores', 'concedes', 'offensive_value', 'defensive_value', "vaep_value", 'exp_scores', 'exp_concedes', 'exp_offensive_value', 'exp_defensive_value', "exp_vaep_value"]], how = "left", left_on = ['Match_ID', 'Chain_Number', 'Order'], right_on=['match_id', 'chain_number', 'order'])
assert chains.shape[0] == number_of_rows
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,ReceiverTeam,Receiver,type_name,outcome_name,shoot,move,goal,end_x,end_y,xT_created,xT_denied,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type1,xScore,xGoals_normalised,match_id,chain_number,order,overall_seconds,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Brisbane Lions,Dayne Zorko,Carry,True,False,True,False,9.0,-6.0,0.005595,NaN,0,1,0,0,0,Centre Bounce,0.0,0.0,202101_BrisbaneLions_Sydney,1.0,2.0,24.0,Hard Ball Get,effective,0.052962,0.009920,0.000000,-0.000000,0.000000,0.246684,0.158121,0.000000,-0.000000,0.000000
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Handball,False,False,True,False,11.0,-7.0,NaN,0.0,0,1,0,0,0,Hard Ball Get,0.0,0.0,202101_BrisbaneLions_Sydney,1.0,3.0,24.0,Handball,ineffective,0.022556,0.050321,-0.030406,-0.040401,-0.070807,0.000000,0.362654,-0.246684,-0.204533,-0.451217
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,Oliver Florent,Carry,True,False,True,False,12.0,-5.0,0.000000,NaN,0,1,0,0,0,Handball,0.0,0.0,202101_BrisbaneLions_Sydney,1.0,4.0,28.0,Loose Ball Get,effective,0.074422,0.008536,0.024101,0.014020,0.038121,0.668677,0.081827,0.306023,-0.081827,0.224196
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,Sydney,George Hewett,Handball,True,False,True,False,14.0,-2.0,0.007561,NaN,0,1,0,0,0,Loose Ball Get,0.0,0.0,202101_BrisbaneLions_Sydney,1.0,5.0,29.0,Handball,effective,0.084299,0.013648,0.009877,-0.005111,0.004766,0.456306,0.099139,-0.212371,-0.017312,-0.229683


Export

In [45]:
chains.to_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/chains_xt_vaep.csv", index=False)